In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL = os.getenv('MODEL')
print(MODEL)



llama3:8b


In [2]:
from langchain_community.llms import Ollama

model = Ollama(model=MODEL)


In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
# chain.invoke("Tell me a joke")


In [4]:
# pdf loader
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("tmt-in-summary-v3.pdf")

pages = loader.load_and_split()

pages



[Document(page_content='TMT in Summary V 3 [20180820]                       \n                                                                      1 \n \n \nส่วนที่ 1 TMT คืออะไร  \nบัญชีข้อมูลยาและรหัสยามาตรฐานไทย ( Thai Medicines Terminology : TMT)  คือ บัญชี\nมาตรฐานของรายการยาและมาตรฐานรหัสยาที่ชี้เฉพาะถึงรายการยาแต่ละรายการ ครอบคลุมรายการยาที่ใช้\nในระบบบริการสุขภาพไทย เป็นมาตรฐานที่สอดคล้องกับมาตรฐานสากล SNOME D-CT (Systematized \nNomenclature Of Medicine Clinical Terms ) โดยมีองค์ประกอบดังภาพ  \n \n \n \nโครงสร้างบัญชีข้อมูลยาและรหัสยามาตรฐานไทย ( TMT Data Model ) ประกอบไปด้วยหน่วย\nความคิดรวบยอดหลัก ( Concepts) เป็นแกนมาตรฐานข้อมูล 8 กลุ่ม ซึ่งในแต่ละกลุ่มข้อมูลจะประกอบไปด้วย\nชุดข้อมูลที่สามารถให้ความหมายอย่างจ าเพาะต่อข้อมูลยาแตกต่างกัน ท าให้สามารถเลือกกลุ่มมาตรฐานข้อมูล\nไปใช้ในหลายวัตถุประสงค์ที่แตกต่างกันในระบบสุขภาพได้อย่างมีประสิทธิภาพ ดังภาพข้างล่างและค าอธิบาย\nในตาราง  \n \n \nTMT โดยสังเขป  V3 \nส่วนที่ 1  TMT คืออะไร  \nส่วนที่ 2  แนวทางและขั้นตอนการด าเนินงานเพื่

In [5]:
# prompt template
from langchain.prompts import PromptTemplate

template = """
Answer the question based only on the following context below.
If you cannot find the information, just output "Sorry, I cannot find the information."

Context: {context}

Question: {question}
"""

prompt = PromptTemplate(template=template)
print(prompt.format(context="Here is some context", question="Here is aquestion"))






Answer the question based only on the following context below.
If you cannot find the information, just output "Sorry, I cannot find the information."

Context: Here is some context

Question: Here is aquestion



In [6]:
chain.input_schema.schema()

{'title': 'OllamaInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'tit

In [9]:
chain = prompt | model | parser

chain.invoke(
    {
        "context": "my name is thanad",
        "question": "what's my name"
    }
)

'Your name is Thanad.'

In [10]:
# docarraymemorysearch
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=MODEL)

vectorstore = DocArrayInMemorySearch.from_documents( 
    pages,
    embedding=embeddings
)

In [11]:
retriver = vectorstore.as_retriever()


In [12]:
from operator import itemgetter

chain = (
    { 
        "context": itemgetter("question") | retriver , 
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

chain.invoke({"question": "What is this document? answer in thai"})


'เอกสารนี้คือ "TMT in Summary V 3" ซึ่งเป็นเอกสารสรุปเกี่ยวกับขั้นตอนและแนวทางในการออกทำรหัส TMT (Thai Medication Terminology)'